# Introduction to Causal Inference

## Randomized Control Trials (A/B Testing)

## Confounding

### Chains

### Forks

### Colliders

### Descendants of Colliders

### Backdoor Criterion

### $do$-calculus

### Front-door Criterion

### Complex Arbitrary DAG with Unmeasured Variables Example

### Mediation & Direct Effects

### Transportability

# Resources


| Image | Notes | Link |
| - | - | - |
| <img src='https://prodimage.images-bn.com/pimages/9781541698963_p0_v1_s600x595.jpg' alt='Book of Why: The New Science of Cause & Effect cover' width=500> | An introduction meant for the more general public. It still is technical, has some math, but focuses more on stories and anecdotes instead of derivations. | [Book of Why: The New Science of Cause & Effect](https://www.amazon.com/Book-Why-Science-Cause-Effect/dp/046509760X) | 
| <img alt='Causal Inference in Statistics: A Primer' src='https://s3.amazonaws.com/vh-woo-images/causal-inference-in-statistics-a-primer-1st-edition.jpg' width=500> | Recommended by Pearl to be read after the Book of Why. Dives more into the math. Has end-of-chapter exercises. *Note: I have the solutions manual! I told Pearl I was self-studying and he graciously gave me a copy!* | [Causal Inference in Statistics: A Primer](https://www.amazon.com/Causal-Inference-Statistics-Judea-Pearl/dp/1119186846) |
| <img src='https://images-na.ssl-images-amazon.com/images/I/511aGcbGLyL._SX343_BO1,204,203,200_.jpg' alt='Causality' width=500> | Goes more in-depth than the Primer book. | [Causality](https://www.amazon.com/Causality-Reasoning-Inference-Judea-Pearl/dp/052189560X) |
| <img alt='Causal Inference: The Mixtape' src='https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1566276665i/47867837._UY630_SR1200,630_.jpg' width=500> | Has a section on DAGs, but focused more on causal inference techniques that are more commonly being used in Economics. *FREELY AVAILABLE*. | [Causal Inference: The Mixtape](https://www.scunning.com/mixtape.html) |
| <img alt='Causal Diagrams: Draw your Assumptions Before Conclusions' src='./img/causal-diagrams-draw-your-assumptions.png' width=500> | *FREE* course on EDX. Makes use of Epidemiological case studies to serve as context as to why drawing your assumptions is important.  | [Causal Diagrams: Draw your Assumptions Before Conclusions](https://online-learning.harvard.edu/course/causal-diagrams-draw-your-assumptions-your-conclusions) |
| not applicable | "Causal Inference: What If" is a book that dives into Hernan & Robins' Potential Outcomes with DAGs approach. *FREE*. | [Causal Inference: What if](https://www.hsph.harvard.edu/miguel-hernan/causal-inference-book/) |